In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [63]:
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.5 MB/s eta 0:00:00


In [3]:
import os
from pathlib import Path
from PIL import Image
import random

os.getcwd()

'/content'

In [4]:
# run shell
git clone https://github.com/huggingface/diffusers.git
cd diffusers
pip install -e .
cd ..
git lfs install
git clone --no-checkout https://huggingface.co/runwayml/stable-diffusion-v1-5
cd stable-diffusion-v1-5
git config core.sparsecheckout true
echo -e 'scheduler\ntext_encoder\ntokenizer\nunet\nvae\nmodel_index.json\n!*.safetensors\n!*.fp16.bin\n!*.non_ema.bin' > .git/info/sparse-checkout
git checkout

Cloning into 'diffusers'...
remote: Enumerating objects: 61850, done.
remote: Counting objects: 100% (1150/1150), done.
remote: Compressing objects: 100% (635/635), done.
remote: Total 61850 (delta 728), reused 743 (delta 444), pack-reused 60700
Receiving objects: 100% (61850/61850), 43.39 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (45043/45043), done.
Git LFS initialized.
Cloning into 'stable-diffusion-v1-5'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 194 (delta 0), reused 0 (delta 0), pack-reused 193 (from 1)
Receiving objects: 100% (194/194), 540.59 KiB | 3.20 MiB/s, done.
Resolving deltas: 100% (68/68), done.
fatal: not in a git directory
/bin/bash: line 1: .git/info/sparse-checkout: No such file or directory
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [55]:
base_path = Path('/content/gdrive/MyDrive/ColabNotebooks/DiffusionModels').resolve()
repo_path = base_path / 'DreamBooth'

if repo_path.exists():
  os.chdir(repo_path)
  !git pull
  os.chdir(base_path)

else:
  os.chdir(base_path)
  !git clone https://github.com/ManuDiehn/DreamBooth.git
  os.chdir(base_path)

print(os.getcwd())

Already up to date.
/content/gdrive/MyDrive/ColabNotebooks/DiffusionModels


In [56]:
SESSION_NAME = "Manu"
WORKSPACE = Path('DreamBooth/output').resolve()
INSTANCE_NAME = SESSION_NAME
OUTPUT_DIR = str(WORKSPACE / "models" / SESSION_NAME)
SESSION_DIR = WORKSPACE / 'Sessions' / SESSION_NAME
INSTANCE_DIR = str(SESSION_DIR / 'instance_images')
IMAGES_FOLDER = (Path('DreamBooth/input/') / SESSION_NAME).resolve()

CONFIG_DIR_ACCELERATE = str(Path('DreamBooth/config/accelerate_config.yaml').resolve())
CONFIG_DIR_TRAINING = str(Path('DreamBooth/config/training_config.yaml').resolve())
SCRIPT_DIR_DREAMBOOTH = str(Path('diffusers/examples/dreambooth/train_dreambooth.py'))

Model_Version = '1.5'
MODEL_NAME = "/content/stable-diffusion-v1-5"

Path(INSTANCE_DIR).mkdir(parents=True, exist_ok=True)

In [57]:
MODEL_NAME

'/content/stable-diffusion-v1-5'

In [58]:
# check if your images have the desired size and copy them
crop_size = 512

for filename in Path(IMAGES_FOLDER).glob('*.png'):
    print(filename)
    new_path_with_file = Path(INSTANCE_DIR) / Path(filename).name
    file = Image.open(filename)
    width, height = file.size
    if file.size != (crop_size, crop_size):
        file = file.resize((crop_size, crop_size), Image.LANCZOS)
    file.save(new_path_with_file, format="PNG")


/content/gdrive/MyDrive/ColabNotebooks/DiffusionModels/DreamBooth/input/Manu/picmd_1.png


In [59]:
training_steps = 350
learning_rate = 1e-6

resolution = 512
precision="fp16" #no

enable_text_encoder_training = True

save_checkpoint_every_n_steps = True
save_checkpoint_every = 500

Seed=random.randint(1, 999999)
instance_prompt = "picmd"


In [60]:
from torch import mode
import yaml
def create_configs(
        config_dir_accelerate,
        config_dir_training,
        model_dir,
        instance_dir,
        output_dir,
        prompt,
        learning_rate,
        seed,
        resolution,
        precision,
        training_steps,
    ):

    config_accelerate = {
      'compute_environment': 'LOCAL_MACHINE',
      'distributed_type': 'NO',
      'num_machines': 1,
      'num_processes': 1,
      'mixed_precision': precision,
    }

    # Write the configuration to a YAML file
    with open(config_dir_accelerate, 'w') as file:
        yaml.safe_dump(config_accelerate, file, default_flow_style=False)

    config_training = {
        'gradient_accumulation_steps': 1,
        'gradient_checkpointing': True,
        'instance_data_dir': instance_dir,
        'instance_prompt': prompt,
        'learning_rate': learning_rate,
        'lr_scheduler': 'linear',
        'lr_warmup_steps': 0,
        'max_train_steps': training_steps,
        'output_dir': output_dir,
        'pretrained_model_name_or_path': model_dir,
        'resolution': resolution,
        'seed': seed,
        'train_batch_size': 1,
        'use_8bit_adam': True,
    }

    return config_training


config_training = create_configs(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_dir_training=CONFIG_DIR_TRAINING,
    model_dir=MODEL_NAME,
    instance_dir=INSTANCE_DIR,
    output_dir=OUTPUT_DIR,
    prompt=instance_prompt,
    learning_rate=learning_rate,
    seed=Seed,
    resolution=resolution,
    precision=precision,
    training_steps=training_steps,
)

In [61]:
config_training

{'gradient_accumulation_steps': 1,
 'gradient_checkpointing': True,
 'instance_data_dir': '/content/gdrive/MyDrive/ColabNotebooks/DiffusionModels/DreamBooth/output/Sessions/Manu/instance_images',
 'instance_prompt': 'picmd',
 'learning_rate': 1e-06,
 'lr_scheduler': 'linear',
 'lr_warmup_steps': 0,
 'max_train_steps': 350,
 'output_dir': '/content/gdrive/MyDrive/ColabNotebooks/DiffusionModels/DreamBooth/output/models/Manu',
 'pretrained_model_name_or_path': '/content/stable-diffusion-v1-5',
 'resolution': 512,
 'seed': 419545,
 'train_batch_size': 1,
 'use_8bit_adam': True}

In [ ]:
def launch_training(config_dir_accelerate, config_training, script_dir):

    command = (
        f"accelerate launch --config_file=\"{config_dir_accelerate}\" \"{script_dir}\" "
        f"--gradient_accumulation_steps {str(config_training['gradient_accumulation_steps'])} "
        f"--gradient_checkpointing "
        f"--instance_data_dir {config_training['instance_data_dir']} "
        f"--instance_prompt \"{config_training['instance_prompt']}\" "
        f"--learning_rate {str(config_training['learning_rate'])} "
        f"--lr_scheduler {config_training['lr_scheduler']} "
        f"--lr_warmup_steps {str(config_training['lr_warmup_steps'])} "
        f"--max_train_steps {str(config_training['max_train_steps'])} "
        f"--output_dir {config_training['output_dir']} "
        f"--pretrained_model_name_or_path \"{config_training['pretrained_model_name_or_path']}\" "
        f"--resolution {str(config_training['resolution'])} "
        f"--seed {str(config_training['seed'])} "
        f"--train_batch_size {str(config_training['train_batch_size'])} "
        f"--use_8bit_adam"
    )

    get_ipython().system(command)

os.chdir("/content/")
launch_training(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_training=config_training,
    script_dir=SCRIPT_DIR_DREAMBOOTH,
)

2024-05-19 20:44:21.170926: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 20:44:21.170970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 20:44:21.172173: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 20:44:22.238025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/19/2024 20:44:23 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of ty